In [1]:
import logging
import os
import sys
import tempfile
import torch
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    RandFlipd,
    Rand2DElasticd,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    RandGaussianNoised
)
from monai.visualize import plot_2d_or_3d_image
import getData
from monai.networks.nets import UNet

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

train_images, train_segs, val_images, val_segs, _, _ = getData.getImageSegTrainValTest("ALL")

train_files = [{"img": img, "seg": seg}
                for img, seg in zip(train_images, train_segs)]
val_files = [{"img": img, "seg": seg}
                for img, seg in zip(val_images, val_segs)]

# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["img", "seg"]),
        ScaleIntensityd(keys=["img", "seg"]),
        RandFlipd(keys=["img", "seg"], prob=0.5, spatial_axis = 1), #mirror in vertical axis
        RandRotate90d(keys=["img", "seg"], prob=0.75, max_k = 3, spatial_axes=[0, 1]),
        Rand2DElasticd(keys=["img", "seg"], prob = 1, spacing = (10,10), magnitude_range = (1,2), padding_mode = "zeros",
            mode = ["bilinear", "nearest"]),
        RandGaussianNoised(keys=["img"], prob = 0.1, mean = 0, std = 0.1),
        EnsureTyped(keys=["img", "seg"]),  
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["img", "seg"]),
        ScaleIntensityd(keys=["img", "seg"]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)



MONAI version: 0.8.0
Numpy version: 1.19.3
Pytorch version: 1.8.2+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.6
Nibabel version: 3.2.2
scikit-image version: 0.19.1
Pillow version: 9.0.1
Tensorboard version: 2.5.0
gdown version: 4.2.1
TorchVision version: 0.9.2+cu111
tqdm version: 4.58.0
lmdb version: 1.3.0
psutil version: 5.8.0
pandas version: 1.1.4
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
# The following training loop is an adaptation of this monai project
# https://github.com/Project-MONAI/tutorials/blob/main/2d_segmentation/torch/unet_training_dict.py
# All metrics calculated from the validation set contain a similar structure

def main(tempdir):

    # create a training data loader
    train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    train_loader = DataLoader(
        train_ds,
        batch_size=4,
        shuffle=True,
        num_workers=4,
        collate_fn=list_data_collate,
        pin_memory=torch.cuda.is_available(),
    )

    # create a validation data loader
    val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=1,
                            num_workers=4, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=False,
                             reduction="mean", get_not_nans=False)
    post_trans = Compose([EnsureType(), Activations(
        sigmoid=True), AsDiscrete(threshold=0.5)])
    # create UNet, DiceLoss and SGD optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)
    loss_function = monai.losses.DiceLoss(smooth_nr = 0, smooth_dr = 1e-5, sigmoid = True)
    optimizer = torch.optim.SGD(model.parameters(),1e-3, momentum = 0.99)

    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter()
    for epoch in range(300):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{300}")
        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            # calculate loss and perform optimization with backpropagation
            step += 1
            inputs, labels = batch_data["img"].to(
                device), batch_data["seg"].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(),
                              epoch_len * epoch + step)
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        
        # calculate dice loss for the validation set
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(
                        device), val_data["seg"].to(device)
                    output = model(val_images)
                    val_outputs = [post_trans(i) for i in decollate_batch(output)]

                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)
                    
                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(),
                               "best_metric_model_segmentation2d_dict.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_dice", metric, epoch + 1)
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label
                plot_2d_or_3d_image(val_images, epoch + 1,
                                    writer, index=0, tag="image")
                plot_2d_or_3d_image(val_labels, epoch + 1,
                                    writer, index=0, tag="label")
                plot_2d_or_3d_image(val_outputs, epoch + 1,
                                    writer, index=0, tag="output")

    print(
        f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
    writer.close()


if __name__ == "__main__":
    with tempfile.TemporaryDirectory() as tempdir:
        main(tempdir)


----------
epoch 1/300
1/56, train_loss: 0.9030
2/56, train_loss: 0.8957
3/56, train_loss: 0.9452
4/56, train_loss: 0.9171
5/56, train_loss: 0.8994
6/56, train_loss: 0.9191
7/56, train_loss: 0.8918
8/56, train_loss: 0.8976
9/56, train_loss: 0.9149
10/56, train_loss: 0.9275
11/56, train_loss: 0.9483
12/56, train_loss: 0.9618
13/56, train_loss: 0.9329
14/56, train_loss: 0.9151
15/56, train_loss: 0.9064
16/56, train_loss: 0.8886
17/56, train_loss: 0.9281
18/56, train_loss: 0.9189
19/56, train_loss: 0.9060
20/56, train_loss: 0.9131
21/56, train_loss: 0.9190
22/56, train_loss: 0.8948
23/56, train_loss: 0.8863
24/56, train_loss: 0.9151
25/56, train_loss: 0.9361
26/56, train_loss: 0.8856
27/56, train_loss: 0.9138
28/56, train_loss: 0.9084
29/56, train_loss: 0.8916
30/56, train_loss: 0.9064
31/56, train_loss: 0.8819
32/56, train_loss: 0.9080
33/56, train_loss: 0.8911
34/56, train_loss: 0.9051
35/56, train_loss: 0.8921
36/56, train_loss: 0.9006
37/56, train_loss: 0.9139
38/56, train_loss: 0.862

KeyboardInterrupt: 

In [4]:
def main(tempdir):
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    _, _, val_images, val_segs, _, _ = getData.getImageSegTrainValTest("0")

    val_files = [{"img": img, "seg": seg} for img, seg in zip(val_images, val_segs)]

    val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=1,
                            num_workers=4, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=False,
                             reduction="mean", get_not_nans=False)
    post_trans = Compose([EnsureType(), Activations(
        sigmoid=True), AsDiscrete(threshold=0.5)])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)

    # load current model 
    model.load_state_dict(torch.load(
        "best_metric_model_segmentation2d_dict.pth"))

    model.eval()
    with torch.no_grad():
        for val_data in val_loader:
            val_images, val_labels = val_data["img"].to(
                device), val_data["seg"].to(device)
            val_output = model(val_images)
            val_outputs = [post_trans(i) for i in decollate_batch(val_output)]
            val_labels = decollate_batch(val_labels)

            # compute metric for current iteration
            dice_metric(y_pred=val_outputs, y=val_labels)

        # aggregate the final mean dice result
        print("evaluation metric:", dice_metric.aggregate().item())
        # reset the status
        dice_metric.reset()


if __name__ == "__main__":
    with tempfile.TemporaryDirectory() as tempdir:
        main(tempdir)

MONAI version: 0.8.0
Numpy version: 1.19.3
Pytorch version: 1.8.2+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.6
Nibabel version: 3.2.2
scikit-image version: 0.19.1
Pillow version: 9.0.1
Tensorboard version: 2.5.0
gdown version: 4.2.1
TorchVision version: 0.9.2+cu111
tqdm version: 4.58.0
lmdb version: 1.3.0
psutil version: 5.8.0
pandas version: 1.1.4
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

evaluation metric: 0.913703203201294
